# Water Balance Model Time Series Runs - LTMs, Variability, E-flows
   
***
<font color=red>Title: Long Term Means, Climate Variability and Environmental Flow indicators over runoff and discharge time series. The original code used water balance model outputs from the CUNY WBM with TerraClimate Climate forcing. Indicators include Coefficient of Variation (CV), Cumulative Distribution Function (CDF), and Environmental Stress (eflow or estress).

Author: Pamela A. Green  
Date: December 31, 2022  

This code hase been modified and updated from the original code developed between January and December 31, 2022 by Pamela Green, Senior Research Associate, CUNY Advanced Science Research Center, New York, NY. Code has been updated to be generically aplicable to any raster water balance model output although end user will need to ensure all raster data are in a consistent resolution and projection. 

The code uses an existing Water Balance Model function from the RGIS model (https://github.com/bmfekete/RGIS) to route runoff along a river network with a simple flow accumulation model. The RGIS functions are run via a command line argument using the python subprocess module. This will require the user to install RGIS on their local system via the GitHub link provided above. Alternatively, user may choose to use a different river network and/or network routing function in place of the WBM RGIS model.</font>
***

Annual and monthly Long Term Means (LTM) are calculated for Pristine and Disturbed runoff and discharge from water balance model runs over the specified time period.

Contemporary Climate Variability indicators were developed using intra- and inter annual variability in available water resources from the WBM-TerraClimate multi-decadal discharge datasets identified above. Coefficient of Variation (CV) indicators were calculated across the time periods specified as (Standard Deviation / Mean). The CV indicators  were then ranked with a cumulative dstribution function (CDF) on a continuous 0-1 scale of low to high climate variability threat identifying areas most vulnerable to large shifts in water availability due to seasonality (intra-annual) and longer term (30-year inter-annual) climate patterns. 

Environmental stress is defined as occuring when the difference between pristine (non-human impacted) flow and disturbed (human impacted) flow exceeds a given threshold (X%). A 20% difference has been used as standard in the past, however, this is now under debate so an option to use different flow thresholds has been provided in the code. The Environmental Stress (ESTRESS) indicator counts the number of months over the time series in which the threshold is exceeded (either, + or -).

Datasets produced:
- Long term mean (LTM) for Pristine and Disturbed Runoff and Discharge over time period
- Climate Variability Coefficient of Variation (CV): Global gridded 6-min resolution climate variability calculated as stardard deviation divided by mean over contemporary time period
- Climate Variability Cumulative istribution Function (CDF): Global gridded 6-min resolution climate variability indicator scores ranked 0-1 for contemporary time period
- ESTRESS: Number of months where Pristine (non-human impacted) flow is X% greater or less than Disturbed (human impacted) flow.

The following steps were carried out to develop the indicators for this analysis:
- Read in Pristine and Disturbed Runoff and Discharge from water balance model outputs
- Aggregating Monthly Data to Annual values
- Creating Coefficient of Variation for Annual water balance model Discharge 
- Creating Coefficient of Variation for Monthly water balance model Discharge (LTM and for each year)
- Created CDF of Annual and Monthly CVs, Inter- and Intra- Annual variability indicators 
- Combine Inter- and Intra- Annual variability indicators into single Climate Vulnerability indicator
- Calculate difference between Pristine and Disturbed Discharge over time frame, count months where limit is +/- exceeded

<b>References for datasets used in this example:</b>

Qin, Y., Abatzoglou, J.T., Siebert, S. et al. Agricultural risks from changing snowmelt. Nat. Clim. Chang. 10, 459–465 (2020). https://doi.org/10.1038/s41558-020-0746-8.

Wisser, D., B. M. Fekete, C. J. Vörösmarty, and A. H. Schumann. “Reconstructing 20th Century Global Hydrography: A Contribution to the Global Terrestrial Network- Hydrology (GTN-H).” Hydrology and Earth System Sciences 14, no. 1 (January 6, 2010): 1–24. https://doi.org/10.5194/hess-14-1-2010.

Fekete, Balázs M., Charles J. Vörösmarty, and Richard B. Lammers. “Scaling Gridded River Networks for Macroscale Hydrology: Development, Analysis, and Control of Error.” Water Resources Research 37, no. 7 (July 2001): 1955–67. https://doi.org/10.1029/2001WR900024.

# Modules and Functions

In [ ]:
import numpy as np
import pandas as pd
import scipy
import rgis as rg

from scipy import stats
from scipy.stats import rankdata
from osgeo import gdal, gdal_array, osr, ogr
from osgeo.gdalconst import *
import scipy.ndimage as nd
import xarray as xr

## Single band numpy array to GeoTiff file function

In [ ]:
def save2File(rA, outname, nrows, ncols, geo_transform):  
    outDs = gdal.GetDriverByName('GTiff').Create(outname, ncols, nrows, 1, gdal.GDT_Float32)
    outBand = outDs.GetRasterBand(1)
    outBand.WriteArray(rA)
    outDs.SetGeoTransform(geo_transform)
    srs = osr.SpatialReference()
    srs.ImportFromEPSG(4326)
    outDs.SetProjection(srs.ExportToWkt())
    outDs = None

    """
    geotransform[0] = top left x
    geotransform[1] = w-e pixel resolution
    geotransform[2] = 0
    geotransform[3] = top left y
    geotransform[4] = 0
    geotransform[5] = n-s pixel resolution (negative value)
    """

## MultiBand numpy array to GeoTiff function

In [ ]:
def CreateMultiGeoTiff(Array, Name, driver, NDV, GeoT, Projection, DataType):
    Array[np.isnan(Array)] = NDV
    DataSet = gdal.GetDriverByName(driver).Create(Name, Array.shape[2], Array.shape[1], Array.shape[0], DataType)
    DataSet.SetGeoTransform(GeoT)
    srs = osr.SpatialReference()
    srs.ImportFromEPSG(Projection)
    DataSet.SetProjection(srs.ExportToWkt() )
    for i, image in enumerate(Array, 1):
        DataSet.GetRasterBand(i).WriteArray( image )
        DataSet.GetRasterBand(i).SetNoDataValue(NDV)
    DataSet.FlushCache()
    return Name

## Rank Cumulative Distribution Function (CDF)

In [ ]:
def rankCDF(array):
    array[np.isnan(array)] = 0    
    rank = np.reshape((rankdata(array, method='dense') - 1), array.shape)
    rank = np.where(array == 0.000, 0.000, rank)
    cdf = rank / np.amax(rank)
    
    return cdf

# Global Variables, Definitions, Datasets

## Define Global Directory Variables; paths on server

In [ ]:
WBMTC_dir = '/Users/pamelagreen/Desktop/Data/TerraClimate/wbm/'
var_outdir = '/Users/pamelagreen/Desktop/Data/TerraClimate/wbm/variability/'

RGISdir='/asrc/ecr/balazs/GHAAS/RGISresults/Global/'

net_dir = '/Users/pamelagreen/Desktop/Data/Griffith2022/RiverBasin/'
cellarea = '/Users/pamelagreen/Desktop/Data/cellarea/CellArea_6m.tif'
upAreafile= WBMTC_dir + 'Global_UpstreamAreakm2_6min.gdbc'
upro_dir = '/asrc/ecr/pamela/TerraClimate/wbm/UpRO/'

n = 300

## Read in River Basin and Subbasin, Cellarea, and Continent grids at 6min resolution

In [ ]:
fin = str(net_dir) + 'HydroSTN06_Subbasin.tif'
ds = gdal.Open(str(fin))
subbas = np.array(ds.GetRasterBand(1).ReadAsArray())

del ds

fin = str(net_dir) + 'Global_Basin_HydroSTN30_06min_GHAAS3.tif'
ds = gdal.Open(str(fin))
basin = np.array(ds.GetRasterBand(1).ReadAsArray())
basin[np.isnan(basin)] = 0

basin_name_file = 'MajorBasinLookUp.xlsx'
pd_basin_name = pd.read_excel(net_dir + basin_name_file, sheet_name='LookupVALUES')
del pd_basin_name['BasinName']
pd_basin_name = pd_basin_name[(pd_basin_name['ID'] != 0)]

fin = str(cellarea)
ds = gdal.Open(str(fin))
CellArea6m = np.array(ds.GetRasterBand(1).ReadAsArray())
CellArea6m[np.isnan(CellArea6m)] = 0

UPAREA = rg.grid(upAreafile)
UPAREA.Load()
UpArea = UPAREA.Data  # .flatten()
UpArea[np.isnan(UpArea)] = 0
UpArea = UpArea[0, :, :]

## Explicitely define projection information for 6min rasters

In [ ]:
ncols6m = 3600
nrows6m = 1500
xll6m = -180
yll6m = -60
xur6m = 180
yur6m = 90
cellsize6m = 0.1
cellang = cellsize6m * -1
nodata = -9999
geotr6m = ([ xll6m, cellsize6m, 0, yur6m, 0, cellang ])

geodriver = 'GTiff'
geoproj = 4326

head6m = 'ncols ' + str(ncols6m) + '\nnrows ' + str(nrows6m) + '\nxllcorner ' + str(xll6m) + '\nyllcorner ' + str(yll6m) + '\ncellsize ' + str(cellsize6m) + '\nNODATA_value ' + str(nodata)

reso = '6min'

## Establish Time Series Variables

Options are to work with Discharge, Runoff (RO, mm/time period), or Runoff Volume (ROV, m3/time period)

In [ ]:
StartYear = 2000
EndYear = 2021
EndYearA = EndYear - 1

#ts_var = 'ROV'
#ts_var = 'RO'
ts_var = 'Discharge'

## Establish actively flowing areas
Masks out nonflowing areas. Based on average upstream runoff less than 3mm/yr occuring across the time domain.

In [ ]:
for x in range(StartYear, EndYear):
    upROfilenmp= upro_dir + '/UpRO' + str(x) + 'p.gdbc'
    UPROp = rg.grid(upROfilenmp)
    UPROp.Load()
    UpROp = UPROp.Data  # .flatten()
    UpROp[np.isnan(UpROp)] = 0
    UpROp = UpROp[0, :, :]
    maskp = np.where(UpROp <= 3.0000, 1, 0)
    if x == StartYear:
        mask_yr = maskp
        maskp_yr = maskp
    else:
        mask_yr = maskp_yr + maskp
        maskp_yr = maskp_yr + maskp

flow_mask = np.where(mask_yr >= 10.0000, 1, 0)
flow_mask = np.resize(flow_mask, (1,nrows6m,ncols6m))

# The WBM ubderperforms in very dry areas and requires manual adjustment for flowing/non-flowing rivers.
# The subbasins below were removed from the actively flowing subset based on review and expert knowledge. 
flow_mask = np.where(subbas == 21599, 1, flow_mask)
flow_mask = np.where(subbas == 21597, 1, flow_mask)
flow_mask = np.where(subbas == 21596, 1, flow_mask)
flow_mask = np.where(subbas == 21595, 1, flow_mask)
flow_mask = np.where(subbas == 21593, 1, flow_mask)
flow_mask = np.where(subbas == 21587, 1, flow_mask)
flow_mask = np.where(subbas == 21583, 1, flow_mask)
flow_mask = np.where(subbas == 21598, 1, flow_mask)

outname = var_outdir + 'UpROmask_year.tif'
save2File(mask_yr[:, :], outname, nrows6m, ncols6m, geotr6m)
outname = var_outdir + 'UpROmaskPRIST_year.tif'
save2File(maskp_yr[:, :], outname, nrows6m, ncols6m, geotr6m)
outname = var_outdir + 'flow_mask.tif'
save2File(flow_mask[0,:, :], outname, nrows6m, ncols6m, geotr6m)

# Read  WBM time series data into XArrays

## Read in Contemporary Annual Time Series Data from RGIS

In [ ]:
modelDIST='TCfull+WBMstableBalanceDist19'
modelPRIST='TCfull+WBMstableBalancePrist'
Meas='RiverDischarge'
res = '06min'
TSfull='Annual'
TSshort='aTS'

filenmDISTAnn= RGISdir + Meas + '/' + modelDIST + '/' + res + '/' + TSfull + '/' + 'Global_' + Meas + '_' + modelDIST + '_' + res + '_' + TSshort
filenmPRISTAnn= RGISdir + Meas + '/' + modelPRIST + '/' + res + '/' + TSfull + '/' + 'Global_' + Meas + '_' + modelPRIST + '_' + res + '_' + TSshort

for x in range(StartYear, EndYear):
    print(str(x) +" ", end='')
    nameDISTAnn = str(filenmDISTAnn) + str(x) + ".gdbc.gz"
    WBMDIST = rg.grid(nameDISTAnn)
    WBMDIST.Load()
    wbmDISTAnn = WBMDIST.Data  # .flatten()
    wbmDISTAnn[np.isnan(wbmDISTAnn)] = 0
    namePRISTAnn = str(filenmPRISTAnn) + str(x) + ".gdbc.gz"
    WBMPRIST = rg.grid(namePRISTAnn)
    WBMPRIST.Load()
    wbmPRISTAnn = WBMPRIST.Data  # .flatten()
    wbmPRISTAnn[np.isnan(wbmPRISTAnn)] = 0
    outname = WBMTC_dir + 'WBM_TerraClimate_OUTPUTQ_DIST_' + str(x) + '.tif'
    save2File(wbmDISTAnn[0, :, :], outname, nrows6m, ncols6m, geotr6m)
    outname = WBMTC_dir + 'WBM_TerraClimate_OUTPUTQ_PRIST_' + str(x) + '.tif'
    save2File(wbmPRISTAnn[0, :, :], outname, nrows6m, ncols6m, geotr6m)
    if x == StartYear:
        array_TS_DIST_ann = wbmDISTAnn        
        array_TS_PRIST_ann = wbmPRISTAnn 
    else:
        array_TS_DIST_ann = np.concatenate(([array_TS_DIST_ann, wbmDISTAnn]), axis=0)
        array_TS_PRIST_ann = np.concatenate(([array_TS_PRIST_ann, wbmPRISTAnn]), axis=0)

tempname = WBMTC_dir + 'WBM_TerraClimate_Discharge_Dist_Ann' + str(StartYear) + '-' + str(EndYearA) + '_' + str(reso) + '.tif'
CreateMultiGeoTiff(array_TS_DIST_ann[:, :, :], tempname, geodriver, nodata, geotr6m, geoproj, gdal.GDT_Float32)
tempname = WBMTC_dir + 'WBM_TerraClimate_Discharge_Prist_Ann' + str(StartYear) + '-' + str(EndYearA) + '_' + str(reso) + '.tif'
CreateMultiGeoTiff(array_TS_PRIST_ann[:, :, :], tempname, geodriver, nodata, geotr6m, geoproj, gdal.GDT_Float32)

print("")
print("Export to GeoTIFF")

## Read in Contemporary Monthly Time Series Data from RGIS

In [ ]:
print("Monthly...")

modelDIST='TCfull+WBMstableBalanceDist19'
modelPRIST='TCfull+WBMstableBalancePrist'
Meas='RiverDischarge'
res = '06min'
TSfull='Monthly'
TSshort='mTS'

filenmDISTMo= RGISdir + Meas + '/' + modelDIST + '/' + res + '/' + TSfull + '/' + 'Global_' + Meas + '_' + modelDIST + '_' + res + '_' + TSshort
filenmPRISTMo= RGISdir + Meas + '/' + modelPRIST + '/' + res + '/' + TSfull + '/' + 'Global_' + Meas + '_' + modelPRIST + '_' + res + '_' + TSshort

for x in range(StartYear, EndYear):
    print(str(x) +" ", end='')
    nameDISTMo = str(filenmDISTMo) + str(x) + ".gdbc.gz"
    WBMDIST = rg.grid(nameDISTMo)
    WBMDIST.Load()
    wbmDISTMo = WBMDIST.Data  # .flatten()
    wbmDISTMo[np.isnan(wbmDISTMo)] = 0
    namePRISTMo = str(filenmPRISTMo) + str(x) + ".gdbc.gz"
    WBMPRIST = rg.grid(namePRISTMo)
    WBMPRIST.Load()
    wbmPRISTMo = WBMPRIST.Data  # .flatten()
    wbmPRISTMo[np.isnan(wbmPRISTMo)] = 0
    tempname = WBMTC_dir + 'WBM_TerraClimate_Discharge_Dist_Monthly_' + str(x) + '.tif'
    CreateMultiGeoTiff(wbmDISTMo[:, :, :], tempname, geodriver, nodata, geotr6m, geoproj, gdal.GDT_Float32)
    tempname = WBMTC_dir + 'WBM_TerraClimate_Discharge_Prist_Monthly_' + str(x) + '.tif'
    CreateMultiGeoTiff(wbmPRISTMo[:, :, :], tempname, geodriver, nodata, geotr6m, geoproj, gdal.GDT_Float32)
    if x == StartYear:
        array_TS_DIST_mo = wbmDISTMo       
        array_TS_PRIST_mo = wbmPRISTMo 
    else:
        array_TS_DIST_mo = np.concatenate(([array_TS_DIST_mo, wbmDISTMo]), axis=0)
        array_TS_PRIST_mo = np.concatenate(([array_TS_PRIST_mo, wbmPRISTMo]), axis=0)


# Annual and Monthly Long Term Mean (LTM), CV and CDF Calculations for contemporary time period

In [ ]:
print("Creating Xarrays..")

lon_tup = np.arange(xll6m, xur6m, cellsize6m) + (cellsize6m / 2)
lat_tup = np.arange(yll6m, yur6m, cellsize6m)
lat_tup = lat_tup[::-1] + (cellsize6m / 2)

num_yrs = EndYear - StartYear
num_mos = num_yrs * 12

start_range = str(StartYear) + "-01-01"
time = pd.date_range(start_range, freq="Y", periods=num_yrs)
time_yr = pd.date_range(start_range, freq="Y", periods=num_yrs)
time_mo = pd.date_range(start_range, freq="M", periods=num_mos)
years = time.strftime('%Y')
months12 = time_mo.to_period('M')
months = time_mo.strftime('%Y-%m')

minflow = 0.003

print("  Annual ")
da_TS_ann_DIST = xr.DataArray(data=array_TS_DIST_ann,
                  coords=[years, lat_tup, lon_tup],
                  dims=["date", "lat", "lon"],
                  name="TS_DIST_ann",
                  attrs=dict(description="WBM-TC Annual DIST", units="cms",)
                  )

TS_ann_DIST_max = da_TS_ann_DIST.max(dim = "date").values
TS_ann_DIST_min = da_TS_ann_DIST.min(dim = "date").values
CV_ann_lowflow = np.where((TS_ann_DIST_max - TS_ann_DIST_min) <= minflow, 1, 0)

TS_ann_DIST_mean = da_TS_ann_DIST.mean(dim = "date").values
TS_ann_DIST_std = da_TS_ann_DIST.std(dim = "date").values
TS_ann_DIST_cv = (da_TS_ann_DIST.std(dim = "date") / da_TS_ann_DIST.mean(dim = "date")).values
TS_ann_DIST_cv_CDF = rankCDF(TS_ann_DIST_cv)

da_TS_ann_PRIST = xr.DataArray(data=array_TS_PRIST_ann,
                  coords=[years, lat_tup, lon_tup],
                  dims=["date", "lat", "lon"],
                  name="TS_PRIST_ann",
                  attrs=dict(description="WBM-TC Annual PRIST", units="cms",)
                  )

TS_ann_PRIST_mean = da_TS_ann_PRIST.mean(dim = "date").values
TS_ann_PRIST_std = da_TS_ann_PRIST.std(dim = "date").values
TS_ann_PRIST_cv = (da_TS_ann_PRIST.std(dim = "date") / da_TS_ann_PRIST.mean(dim = "date")).values
TS_ann_PRIST_cv_CDF = rankCDF(TS_ann_PRIST_cv)

print("  Monthly ")
da_TS_mo_DIST = xr.DataArray(data=array_TS_DIST_mo,
                  coords=[time_mo, lat_tup, lon_tup],
                  dims=["date", "lat", "lon"],
                  name="array_TS_DIST_mo",
                  attrs=dict(description="TC DIST Monthly", units="cms",)
                  )

TS_mo_DIST_mean = da_TS_mo_DIST.mean(dim = "date").values
TS_mo_DIST_std = da_TS_mo_DIST.std(dim = "date").values
TS_mo_DIST_cv = (da_TS_mo_DIST.std(dim = "date") / da_TS_mo_DIST.mean(dim = "date")).values
TS_mo_DIST_cv_CDF = rankCDF(TS_mo_DIST_cv)

da_TS_mo_PRIST = xr.DataArray(data=array_TS_PRIST_mo,
                  coords=[time_mo, lat_tup, lon_tup],
                  dims=["date", "lat", "lon"],
                  name="array_TS_PRIST_mo",
                  attrs=dict(description="TC PRIST Monthly", units="cms",)
                  )
TS_mo_PRIST_mean = da_TS_mo_PRIST.mean(dim = "date").values
TS_mo_PRIST_std = da_TS_mo_PRIST.std(dim = "date").values
TS_mo_PRIST_cv = (da_TS_mo_PRIST.std(dim = "date") / da_TS_mo_PRIST.mean(dim = "date")).values
TS_mo_PRIST_cv_CDF = rankCDF(TS_mo_PRIST_cv)

print("  Monthly LTM")
da_TS_moLTM_DIST = xr.DataArray(data=array_TS_DIST_mo,
                  coords=[time_mo, lat_tup, lon_tup],
                  dims=["date", "lat", "lon"],
                  name="array_TS_DIST_mo",
                  attrs=dict(description="TC DIST Monthly", units="cms",)
                  )

TS_moLTM_DIST_mean = da_TS_moLTM_DIST.groupby("date.month").mean()
TS_moLTM_DIST_cv = stats.variation(TS_moLTM_DIST_mean, axis=0)
TS_moLTM_DIST_cv_CDF = rankCDF(TS_moLTM_DIST_cv)

da_TS_moLTM_PRIST = xr.DataArray(data=array_TS_PRIST_mo,
                  coords=[time_mo, lat_tup, lon_tup],
                  dims=["date", "lat", "lon"],
                  name="array_TS_PRIST_mo",
                  attrs=dict(description="TC PRIST Monthly", units="cms",)
                  )
TS_moLTM_PRIST_mean = da_TS_moLTM_PRIST.groupby("date.month").mean()
TS_moLTM_PRIST_cv = stats.variation(TS_moLTM_PRIST_mean, axis=0)
TS_moLTM_PRIST_cv_CDF = rankCDF(TS_moLTM_PRIST_cv)

print("  Monthly Across Years")
TS_mo_by_yr_DIST_mean = da_TS_moLTM_DIST.groupby("date.year").mean()
TS_mo_by_yr_DIST_std = da_TS_moLTM_DIST.groupby("date.year").std()
TS_mo_by_yr_DIST_cv = (TS_mo_by_yr_DIST_std / TS_mo_by_yr_DIST_mean).values
TS_mo_by_yr_DIST_maxCV = np.amax(TS_mo_by_yr_DIST_cv, axis=0)
TS_mo_by_yr_DIST_meanCV = np.mean(TS_mo_by_yr_DIST_cv, axis=0)

print("Annual LTM OUTPUTQ_DIST & PRIST..")
distqvol_mean = da_TS_ann_DIST.mean(dim = "date").values
pristqvol_mean = da_TS_ann_PRIST.mean(dim = "date").values

outname = WBMTC_dir + 'WBM_TerraClimate_OUTPUTQ_DIST_aLTM_2000-2020.tif'
save2File(distqvol_mean[:, :], outname, nrows6m, ncols6m, geotr6m)
outname = WBMTC_dir + 'WBM_TerraClimate_OUTPUTQ_PRIST_aLTM_2000-2020.tif'
save2File(pristqvol_mean[:, :], outname, nrows6m, ncols6m, geotr6m)

WBMdistQ_mo = da_TS_mo_DIST.groupby("date.month").mean()
npWBMdistQ_mo = WBMdistQ_mo.to_numpy()
WBMpristQ_mo = da_TS_mo_PRIST.groupby("date.month").mean()
npWBMpristQ_mo = WBMpristQ_mo.to_numpy()

outname = WBMTC_dir + 'WBM_TerraClimate_OUTPUTQ_DIST_mLTM_2000-2020.tif'
CreateMultiGeoTiff(npWBMdistQ_mo[:, :, :], outname, geodriver, nodata, geotr6m, geoproj, gdal.GDT_Float32)
outname = WBMTC_dir + 'WBM_TerraClimate_OUTPUTQ_PRIST_mLTM_2000-2020.tif'
CreateMultiGeoTiff(npWBMpristQ_mo[:, :, :], outname, geodriver, nodata, geotr6m, geoproj, gdal.GDT_Float32)

print("Export CVs")
outname = var_outdir + 'WBM_TerraClimate' + str(StartYear) + '-' + str(EndYearA) + '_Q_DIST_CV_InterAnnual_' + str(reso) + '.tif'
save2File(TS_ann_DIST_cv[:, :], outname, nrows6m, ncols6m, geotr6m)
outname = var_outdir + 'WBM_TerraClimate' + str(StartYear) + '-' + str(EndYearA) +'_Q_DIST_CV_IntraAnnual_' + str(reso) + '.tif'
save2File(TS_mo_DIST_cv[:, :], outname, nrows6m, ncols6m, geotr6m)
outname = var_outdir + 'WBM_TerraClimate' + str(StartYear) + '-' + str(EndYearA) +'_Q_LTM_DIST_CV_IntraAnnual_' + str(reso) + '.tif'
save2File(TS_moLTM_DIST_cv[:, :], outname, nrows6m, ncols6m, geotr6m)

outname = var_outdir + 'WBM_TerraClimate' + str(StartYear) + '-' + str(EndYearA) +'_Q_PRIST_CV_InterAnnual_' + str(reso) + '.tif'
save2File(TS_ann_PRIST_cv[:, :], outname, nrows6m, ncols6m, geotr6m)
outname = var_outdir + 'WBM_TerraClimate' + str(StartYear) + '-' + str(EndYearA) +'_Q_PRIST_CV_IntraAnnual_' + str(reso) + '.tif'
save2File(TS_mo_PRIST_cv[:, :], outname, nrows6m, ncols6m, geotr6m)
outname = var_outdir + 'WBM_TerraClimate' + str(StartYear) + '-' + str(EndYearA) +'_Q_LTM_PRIST_CV_IntraAnnual_' + str(reso) + '.tif'
save2File(TS_moLTM_PRIST_cv[:, :], outname, nrows6m, ncols6m, geotr6m)

outname = var_outdir + 'WBM_TerraClimate' + str(StartYear) + '-' + str(EndYearA) +'_Q_DIST_maxCV_IntraAnnual_' + str(reso) + '.tif'
save2File(TS_mo_by_yr_DIST_maxCV[:, :], outname, nrows6m, ncols6m, geotr6m)
outname = var_outdir + 'WBM_TerraClimate' + str(StartYear) + '-' + str(EndYearA) +'_Q_DIST_meanCV_IntraAnnual_' + str(reso) + '.tif'
save2File(TS_mo_by_yr_DIST_meanCV[:, :], outname, nrows6m, ncols6m, geotr6m)

## Below is left in for legacy since "combining" annual and LTM-montly CDF was previously used: however,
## It's best to simply use the monthly CDF over the entire time series if you are looking to capture monhtly and annual variability signals in one indicator.
## You can still use the annual CDF if you are only looking for an inter-annual variability metric (i.e., how do annual patterns vary year to year for long-term trends).
## And you can us the LTM monthly variabilities to compare general patterns of intra-annual variability across different WBM runs or time periods (i.e., how do seasonal patterns change across time periods?).

print("Combining Inter- and Intra- annual statistics..")
TS_DIST_CV_mean = (TS_mo_DIST_cv_CDF + TS_ann_DIST_cv_CDF) / 2
TS_DIST_CV_CDF = rankCDF(TS_DIST_CV_mean)

TS_PRIST_CV_mean = (TS_mo_PRIST_cv_CDF + TS_ann_PRIST_cv_CDF) / 2
TS_PRIST_CV_CDF = rankCDF(TS_PRIST_CV_mean)

TS_DIST_LTM_CV_mean = (TS_moLTM_DIST_cv_CDF + TS_ann_DIST_cv_CDF) / 2
TS_DIST_LTM_CV_CDF = rankCDF(TS_DIST_LTM_CV_mean)

TS_PRIST_LTM_CV_mean = (TS_moLTM_PRIST_cv_CDF + TS_ann_PRIST_cv_CDF) / 2
TS_PRIST_LTM_CV_CDF = rankCDF(TS_PRIST_LTM_CV_mean)

print("Export CDFs")
outname = var_outdir + 'WBM_TerraClimate' + str(StartYear) + '-' + str(EndYearA) +'_Q_DIST_CDF_InterAnnual_' + str(reso) + '.tif'
save2File(TS_ann_DIST_cv_CDF[:, :], outname, nrows6m, ncols6m, geotr6m)
outname = var_outdir + 'WBM_TerraClimate' + str(StartYear) + '-' + str(EndYearA) +'_Q_DIST_CDF_IntraAnnual_' + str(reso) + '.tif'
save2File(TS_mo_DIST_cv_CDF[:, :], outname, nrows6m, ncols6m, geotr6m)
outname = var_outdir + 'WBM_TerraClimate' + str(StartYear) + '-' + str(EndYearA) +'_Q_LTM_DIST_CDF_IntraAnnual_' + str(reso) + '.tif'
save2File(TS_moLTM_DIST_cv_CDF[:, :], outname, nrows6m, ncols6m, geotr6m)

outname = var_outdir + 'WBM_TerraClimate' + str(StartYear) + '-' + str(EndYearA) +'_Q_PRIST_CDF_InterAnnual_' + str(reso) + '.tif'
save2File(TS_ann_PRIST_cv_CDF[:, :], outname, nrows6m, ncols6m, geotr6m)
outname = var_outdir + 'WBM_TerraClimate' + str(StartYear) + '-' + str(EndYearA) +'_Q_PRIST_CDF_IntraAnnual_' + str(reso) + '.tif'
save2File(TS_mo_PRIST_cv_CDF[:, :], outname, nrows6m, ncols6m, geotr6m)
outname = var_outdir + 'WBM_TerraClimate' + str(StartYear) + '-' + str(EndYearA) +'_Q_LTM_PRIST_CDF_IntraAnnual_' + str(reso) + '.tif'
save2File(TS_moLTM_PRIST_cv_CDF[:, :], outname, nrows6m, ncols6m, geotr6m)

outname = var_outdir + 'WBM_TerraClimate' + str(StartYear) + '-' + str(EndYearA) +'_Q_DIST_CDF_InterIntra_' + str(reso) + '.tif'
save2File(TS_DIST_CV_CDF[:, :], outname, nrows6m, ncols6m, geotr6m)
outname = var_outdir + 'WBM_TerraClimate' + str(StartYear) + '-' + str(EndYearA) +'_Q_PRIST_CDF_InterIntra_' + str(reso) + '.tif'
save2File(TS_PRIST_CV_CDF[:, :], outname, nrows6m, ncols6m, geotr6m)
outname = var_outdir + 'WBM_TerraClimate' + str(StartYear) + '-' + str(EndYearA) +'_Q_LTM_DIST_CDF_InterIntra_' + str(reso) + '.tif'
save2File(TS_DIST_LTM_CV_CDF[:, :], outname, nrows6m, ncols6m, geotr6m)
outname = var_outdir + 'WBM_TerraClimate' + str(StartYear) + '-' + str(EndYearA) +'_Q_LTM_PRIST_CDF_InterIntra_' + str(reso) + '.tif'
save2File(TS_PRIST_LTM_CV_CDF[:, :], outname, nrows6m, ncols6m, geotr6m)


# Environmental Flow Analysis

Environmental stress is defined as occuring when the difference between pristine (non-human impacted) flow and disturbed (human impacted) flow exceeds a given threshold (X%). A 20% difference has been used as standard in the past, however, this is now under debate so an option to use different flow thresholds has been provided in the code. The Environmental Stress (ESTRESS) indicator counts the number of months over the time series in which the threshold is exceeded (either, + or -).

The code masks out upstream headwater areas (defined by min upstream area limit) that have modelled irrigation depths below a given limit (50% quantile value of irrigation depth of small headwater cells). This mask is applied to eliminate noise in the modelled data associated with very small irrigation and discharge values.

In [ ]:
## Read in net irrigaton water demand in mm/yr and define 50% irrigation demand quantile (~3.5mm) of small headwater cells for masking

UpAreaLim = 250

filepath = WBMTC_dir + 'WBM_TerraClimate_OUTPUT_IrrNetWD_mm_aLTM_2000-2020.tif'
dataset = gdal.Open(filepath)
irrromm_mean = dataset.ReadAsArray()

irrromm_headw = np.where(UpArea > UpAreaLim, irrromm_mean, 0)
irrmm_flat = irrromm_headw.reshape(-1)
df_mm = pd.DataFrame(irrmm_flat, columns = ['IrrNetmm'])
df_mm = df_mm.dropna()
df_mm_flat = df_mm.loc[~(df_mm == 0).any(axis=1)]

IrrLim = df_mm_flat.IrrNetmm.quantile(0.5)
#75% = 0.1; 34.02mm
#50% = 0.01; 3.681mm
#25% = 0.001; 0.373mm
#10% = 0.0001; 0.04 mm

###################################################################
# ESTRESS = where the difference between Pristine and 
#    Disturbed Discharge (absfrac) exceeds 10, 20, or 30% limit
# absfrac = abs(Pristine_LTM_Q - Disturbed_LTM_Q) / Pristine_LTM_Q 
###################################################################

diff = npWBMpristQ_mo - npWBMdistQ_mo
absdiff = np.absolute(diff)
absfrac = np.where(absdiff[:, :] > 0.0000000, np.where(npWBMpristQ_mo == 0.0, 1, absdiff[:, :] / WBMpristQ_mo), 0.0)

chglist = [10.0, 20.0, 30.0]
for x in chglist:
    chg = x / 100
    if chg == 0.1:
        chg_nm = 'Chg10'
    if chg == 0.2:
        chg_nm = 'Chg20'
    if chg == 0.3:
        chg_nm = 'Chg30'

    print("Create ESTRESS rasters from LTM for " + chg_nm)
    estress = np.where(flow_mask == 1.0, 0.0, np.where(absfrac[:, :] >= chg, 1.0, 0.0))
    
    estressLTM = np.sum(estress, axis = 0)
    estressLTM = np.where(flow_mask == 0, estressLTM, np.NaN)
    estressLTM = estressLTM[0,:,:]
    
    estressLTM = np.where(flow_mask == 0, np.where(UpArea > UpAreaLim, estressLTM, np.where(irrromm_mean > IrrLim, estressLTM, 0)), np.NaN)

    nans = np.isnan(estressLTM)
    
    estressLTM_CNTnonans = np.where(nans, 0.0, 1.0)
    estressLTM_nonans = np.where(nans, 0.0, estressLTM)

    outname = var_outdir + '/Estress_WBM_TerraClimate_2000-2020_LTM_' + chg_nm + '.tif'
    save2File(estressLTM[0, :, :], outname, nrows6m, ncols6m, geotr6m)
    
    outname = var_outdir + '/Estress_WBM_TerraClimate_2000-2020_LTM_CNT_' + chg_nm + '.tif'
    save2File(estressLTM_CNTnonans[0, :, :], outname, nrows6m, ncols6m, geotr6m)
    
    outname = var_outdir + '/Estress_WBM_TerraClimate_2000-2020_LTM_NONANS_' + chg_nm + '.tif'
    save2File(estressLTM_nonans[0, :, :], outname, nrows6m, ncols6m, geotr6m)